### En este notebook voy a intentar clasificar las ubicaciones entre validas o no

In [1]:
import pandas as pd
import re
import numpy as np
import pickle
import nltk
import xgboost as xgb
import matplotlib.pyplot as plt
import lightgbm as lgb
from catboost import CatBoostRegressor
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import STOPWORDS
%matplotlib inline
import spacy
import geopandas
from geopy.geocoders import Nominatim
#requerido
#pip install geopandas
#pip install geopy

#nltk.download('punkt')


In [2]:
## voy a intentar agregar un feature que contenga las coordenadas de la location si la misma es valdia

In [3]:
train = pd.read_csv('train_loc_spacy.csv')
test = pd.read_csv('test_loc_spacy.csv')

In [4]:
train.sample(3)

,id,keyword,location,text,target,loc_nula,loc_valida
950,1376,blown%20up,NaN,@thebriankrause leos ass just got metaphorical...,0,1,0
3339,4783,evacuated,"Nashville, TN",Just got evacuated from the movie theatre for ...,1,0,1
7506,10736,wreckage,Tokyo,Wreckage Confirmed As Part of Missing Malaysia...,1,0,1


In [5]:
locator = Nominatim( user_agent = "myGeocoder")
location = locator.geocode("Villa Ballester")

In [6]:
type(location)

geopy.location.Location

In [7]:
print (location.latitude, location.longitude)

-34.5483812 -58.5564925


In [8]:
location

Location(Villa Ballester, Partido de General San Martín, Buenos Aires, Argentina, (-34.5483812, -58.5564925, 0.0))

In [9]:
location.point

Point(-34.5483812, -58.5564925, 0.0)

In [10]:
def calcular_corr(x, locator):
    loc = locator.geocode(x)
    if loc is None:
            return (np.nan, np.nan)
    return (loc.latitude, loc.longitude)

In [11]:
#### no puedo mandar los 7k de tweets de una porque el sv de GeoCoder me desconecta
#### intento mandar de a tandas de a 500

In [12]:
lista = []
locations = train.location
validas = train.loc_valida
locations[1787] = 'florida' #evito que crashee el locator
for i in range (0,len(locations) ):
    coor = (np.nan, np.nan )
    if validas[i]:
        print(locations[i],i)
        coor = calcular_corr(locations[i],locator)
    lista.append( coor )

/home/riedel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Birmingham 31
Philadelphia, PA 34
London, UK 35
Pretoria 36
Paranaque City 39
GREENSBORO,NORTH CAROLINA 46
England. 49
Sheffield Township, Ohio 50
India 51
Barbados 52
Anaheim 53
USA 55
South Africa 56
Sao Paulo, Brazil 57
Edmonton, Alberta - Treaty 6 59
Concord, CA 63
Calgary, AB 64
Birmingham 65
San Francisco 66
Nashville, TN 68
Santa Clara, CA 69
UK 70
St. Louis, MO 71
Walker County, Alabama 72
Australia 73
North Carolina 74
Norf Carolina 76
San Mateo County, CA 77
North Carolina 78
Njoro, Kenya 79
Arlington, TX 83
South Bloomfield, OH 84
New Hanover County, NC 86
Maldives 87
Manchester, NH 88
Wilmington, NC 89
New Hanover County, NC 91
Alberta | Sask. | Montana 94
Charlotte 95
Baton Rouge, LA 96
Hagerstown, MD 97
Gloucestershire , UK 98
UK 100
Nairobi, Kenya 101
Switzerland 104
US 106
Belgium 113
Switzerland 114
US 115
Switzerland 118
Switzerland 119
Oshawa, Canada 120
Baker City Oregon 121
United States 124
marysville ca  127
Hermosa Beach, CA 129
Pennsylvania 137
Salt Lake City, 

Chicago, IL 1091
Des Moines, IA 1092
Dundas, Ontario 1093
New York 1096
Netherlands 1099
DaKounty, Pa 1100
Tokyo 1102
DÌ_sseldorf, Germany 1103
The land of New Jersey.  1110
Atlanta Georgia 1111
Kabul, Tuebingen, Innsbruck 1114
Nairobi, Kenya  1116
Erbil 1119
Stockton on tees Teesside UK 1120
Screwston, TX 1121
melbourne 1124
Warwick, RI @Dollarocracy also 1128
Sydney, Australia 1135
Overland Park, KS 1145
Puerto Rico 1147
Toronto-Citizen of Canada & US 1148
Washington, DC 1152
Washington, D.C. 1155
Singapore 1156
London 1158
United States 1159
Mumbai 1161
New Delhi, Delhi 1164
Buscame EL tu Melte 1165
Wiltshire 1167
Leicester 1171
Mumbai , India 1172
Pittsburgh PA 1174
Boston 1176
Leeds, England 1178
US 1179
California 1181
UK 1182
Nigeria 1183
Playa del Carmen, Mexico 1184
Leicester 1190
NY, CT & Greece 1192
Mumbai 1194
Mackay, QLD, Australia 1197
NJ 1199
Madison, GA 1201
Denver, Colorado 1205
San Francisco 1206
Chicago, IL 1210
Savannah, GA 1220
dallas 1221
New Orleans ,Louisiana 12

Rockville, Maryland 2015
Australia 2018
Bhopal, Madhya Pradesh, India. 2021
Tennessee 2022
San Jose, California 2028
Loughborough, England 2030
Brooklyn, NY 2032
Atlanta Georgia  2035
Atlanta Georgia  2040
Boston, MA 2041
Newcastle Upon Tyne, England 2044
Atlanta Georgia  2045
Atlanta Georgia  2046
San Jose, CA 2051
Israel 2053
Kansas City, MO 2055
Silver Spring, MD 2056
Jersey - C.I 2057
Afghanistan 2060
Sweden 2061
London 2064
Orlando, FL 2065
Glasgow 2069
somewhere in Portugal 2072
dundalk ireland 2074
Sochi, KDA, RU 2077
Kettering, OH 2081
United States 2082
United States 2087
New York 2095
Buenos Aires, Argentina 2096
The UK 2100
Kensington, MD 2104
Sylacauga, Alabama 2107
USA 2112
Cyprus 2114
kansas 2115
Los Angeles,CA, USA 2116
Portland, OR 2117
New York, New York 2120
Chicago 2123
Alicante, Spain 2124
Mooresville, NC 2127
Mooseknuckle, Maine 2128
Palermo, Sicily 2129
Voorhees, NJ 2132
UK 2136
Atlanta, GA 2137
Lagos 2140
Phoenix, AZ 2144
Edinburgh 2146
Blackpool 2151
the void, U

Pennsylvania, USA 2989
San Diego, California 2990
Idaho 2991
Beirut, Lebanon 2992
El Paso, Texas 2994
Lubbock, TX 2996
Marrakech MÌ©dina, Marrakech - Tensift - Al Haouz 3005
Ellensburg to Spokane 3006
Near Yosemite 3017
Atlanta, GA 3020
Atlanta, GA 3023
chicago 3024
Sydney 3030
California, USA 3033
oklahoma 3034
Desde Republica Argentina 3036
California, USA 3037
Oklahoma City, OK 3039
California, USA 3042
Melbourne, Australia 3044
Barcelona, Spain 3045
Hawaii, USA 3046
New Zealand 3047
Alaska, USA 3049
Okuma Town, Fukushima 3052
Desde Republica Argentina 3057
London 3062
Seattle, WA 3063
Dalkeith, Scotland 3072
Naperville 3074
Michigan  3084
Kutztown, PA 3089
London 3090
Budapest, Hungary 3091
Texas, USA 3092
Houston, TX 3094
Cairo, Egypt. 3095
South Africa 3096
Karachi Pakistan 3099
New York 3105
Edinburgh 3106
USA 3109
not so cool KY 3110
North Carolina 3112
Hampshire UK 3114
South West, England 3116
Redondo Beach, CA 3119
Newcastle, England 3120
nyc 3121
Karachi Pakistan 3124
Mumba

Baltimore, MD 3906
East Lansing, MI 3908
New York 3910
boston 3911
United States 3912
New York 3913
New York 3914
Warrandyte, Australia 3915
British Columbia, Canada 3916
Philadelphia 3920
New York 3921
New York 3923
New York 3924
Niagara Falls, Ontario 3925
USA 3926
New York 3929
New York 3930
New York 3933
USA 3934
United States 3935
New York 3936
New York 3937
New York 3938
United States 3939
New York 3941
USA 3942
Knoxville, TN 3944
New York 3945
Rock Springs, WY 3946
Vietnam 3947
Tampa-St. Petersburg, FL 3948
USA 3950
Huntsville, Alabama 3951
Nova Scotia, Canada 3952
Jakarta/Kuala Lumpur/S'pore 3957
UK 3958
Vietnam 3960
Honolulu, Hawaii 3963
Miami Beach, Fl 3965
Republic of Texas 3966
California, USA 3967
Huntsville AL 3968
West Midlands 3971
Jakarta/Kuala Lumpur/S'pore 3972
Jakarta/Kuala Lumpur/S'pore 3976
Ocean City, NJ 3977
belleville 3979
Kualar Lumpur, Malaysia 3980
Austin, TX 3981
Nadiad ,Gujarat , India!! 3983
South of D.C. 3984
Adventist - Lesson Sabbath 3986
Australia 398

New York 4817
Los Angeles 4819
Leicester, England 4822
Melbourne, Australia 4824
 Queensland, Australia 4829
New Sweden 4835
Chattanooga TN 4839
Nashville, TN 4840
Birmingham, England 4841
Huntsville, AL 4842
Auckland 4844
Seattle 4847
USA 4850
California, USA 4851
San Diego, CA 4856
Tennessee, USA 4859
Huntsville, AL 4863
Hemel Hempstead 4864
Fresno 4866
West Hollywood, CA 4868
Tama, Iowa 4874
Dundee 4876
Minneapolis - St. Paul 4879
Las Vegas aka Hell 4880
Ashburn, VA 4882
St. Louis, Mo 4883
Vancouver Canada 4884
Colorado 4887
Kingston, Jamaica 4888
Nottingham, England 4892
Vancouver, BC, Canada 4896
Ireland 4898
Ecuador 4899
London, UK 4900
London 4910
India 4911
Norway 4912
Wausau, Wisconsin 4914
Orlando, FL 4916
Raleigh, NC 4917
Boston, MA 4919
North Dartmouth, Massachusetts 4923
Lynchburg, VA 4924
Boston, MA 4926
?? Made in the Philippines ?? 4929
Boston/Montreal  4930
Manavadar, Gujarat 4931
Pueblo, Colorado 4932
PG County, MD 4933
London, UK 4939
Detroit, Michigan 4941
Cleveland

St. Louis, MO 5826
United States 5827
Spain - China - Latin America. 5828
Chester, IL 5829
West Africa 5830
Calgary, AB, Canada 5831
California 5832
Made Here In Detroit  5833
Portland, OR 5834
Dallas, Tejas 5836
ATLANTA , GEORGIA  5837
Nashville, TN 5839
London / Birmingham 5842
texas 5843
Yulee, FL 5844
sheffield // rotherham 5851
Wailuku, Maui 5852
Lima, OH 5854
Long Beach, CA 5855
Belfast 5856
New York - Connecticut 5858
Winnipeg, Manitoba 5863
Boston 5865
Florida Forever 5866
Sacramento, CA 5867
Des Moines, IA 5873
Scituate, MA 5875
USA 5876
United States 5878
United States 5880
USA 5881
US 5883
USA 5884
USA 5885
Hamilton County, IN 5887
United States 5888
USA 5889
USA 5890
USA 5891
Dakar 5893
United States 5894
USA 5895
USA 5896
USA 5897
USA 5898
USA 5900
North Carolina, USA 5902
USA 5903
USA 5904
USA 5905
USA 5907
The Sanctuary Network, Rome 5908
USA 5911
San Diego, CA 5918
sneaking glances at Thancred 5921
Charlotte, North Carolina 5923
ny 5928
Trumann, Arkansas 5930
North Caro

Savage States of America 7018
Calgary, AB, Canada 7019
USA 7022
Tema,Accra 7024
phuket thailand 7025
USA 7030
Houston, TX 7031
New York  7035
Santiago,RepÌ¼blica Dominicana 7036
Wilmington, Delaware 7040
Washington, DC 7044
india 7045
anzio,italy 7047
NYC :) Ex- #Islamophobe 7052
Dhaka 7053
Halifax, Nova Scotia 7054
Deadend, UK 7055
Oregon 7056
Washington, D.C. 7057
Jamaica 7060
USA 7061
Auckland, New Zealand 7065
NYC 7066
Scotts Valley, CA 7067
Oregon 7068
Wisconsin, USA 7073
East Aurora, NY 7075
London, UK 7078
Costa Rica 7079
Hamilton, Ontario CA 7080
maryland 7081
Atlanta, GA 7083
INDIA 7084
Milwaukee, WI 7085
CPT & JHB, South Africa 7087
London, UK 7088
Oregon and Washington 7089
Karachi 7090
Connecticut 7091
Perth, Western Australia 7092
UK 7093
Watertown, Mass. 7095
Toronto 7096
Chicago, IL 7102
South Africa 7109
Costa Rica 7110
Newcastle, OK 7114
United Kingdom 7115
Oshawa/Toronto 7117
Very SW CA, USA....Draenor 7118
New Zealand 7120
Kenya 7121
Barbados 7123
Perth, Australia 71

In [13]:
train['coordenadas'] = lista

In [14]:
train.to_csv("csv/train_con_geo.csv")

In [16]:
train.sample(5)

,id,keyword,location,text,target,loc_nula,loc_valida,coordenadas
4487,6383,hostages,The Universe,Cont'd- #Sinjar: referring to a 40-pg document...,1,0,0,"(nan, nan)"
139,199,airplane%20accident,"Palo Alto, CA",Experts in France begin examining airplane deb...,1,0,1,"(37.4443293, -122.1598465)"
1388,2001,bush%20fires,NaN,So apparently there were bush fires near where...,1,1,0,"(nan, nan)"
3858,5488,flames,"Sioux Falls, S.D.",RP said they can see smoke coming from the sil...,1,0,1,"(43.5499749, -96.700327)"
408,588,arson,"Menasha, WI",Arson suspect linked to 30 fires caught in Nor...,1,0,1,"(44.2022293, -88.4465361)"


In [21]:
locations[1427]

'AZ ? TX ? CA ? IL ?'

In [ ]:
#ahora aplico al test
lista = []
locations = test.location
validas = test.loc_valida
locations[1427] = 'US'
for i in range (0,len(locations) ):
    coor = (np.nan, np.nan )
    if validas[i]:
        print(locations[i],i)
        coor = calcular_corr(locations[i],locator)
    lista.append( coor )

/home/riedel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


London 15
Los Angeles, Califnordia 19
Whoop Ass, Georgia 22
India 23
scarborough, ontario 25
Homewood, PA 29
USA 32
Massachusetts 33
Cowtown, Caliii !! 37
All Motorways, UK 39
Frankfurt, Germany 40
Las Vegas  43
Midland, Mi 44
Switzerland 47
California 48
Detroit, MI 52
Switzerland 53
United Kingdom 56
London, England 57
Runcorn 59
Lake Charles, LA 60
California, USA 61
Eagle Pass, Texas 64
New York 69
France 74
Istanbul, TÌ_rkiye 75
High in Prague with Aya 76
Atlanta, GA 79
USA  80
West Chester, PA 85
Chorley, Lancashire, UK 86
San Diego 88
New Orleans 89
eileenborut,webster, tx 90
Salem, MA 101
Chicago, Illinois 102
Colorado 106
United States 107
Plovdiv, Bulgaria 109
California 110
Rhode Island, USA 111
U.S.A. 114
Arizona 115
Wayne, NJ 116
Chicago 117
Wheeler,Wis. 121
Memphis, TN 126
DMZ, AZ 127
Akron, Ohio  130
Irvine, CA 138
Arlington 140
Phuket 144
West Covina CA 147
Campinas Sp 150
New York 153
Bowling Green, Ky 158
Logansport, Indiana 162
Rexburg ID 163
Mexico! ^_^ 165
Los Ange

Las Vegas 1053
Edmonton, Alberta 1054
Washington, DC 1055
District of Columbia, USA 1056
Waterloo, Ontario, Canada 1058
st. louis, missouri 1059
Washington, DC 1060
Alexandria, VA, USA 1061
Woodbridge, VA  1062
Chicago, IL 1065
Chicago, IL 1067
Chicago 1068
Mumbai 1069
India 1074
Oakland, Ca 1075
Twin Falls, Idaho, 83301 1076
Michigander, USA 1078
Texas 1080
Chicago 1082
Florida 1088
Birmingham 1089
Los Angeles, CA 1090
Raleigh, NC 1091
Roanoke, VA 1094
USA,Washington,Seattle 1096
Quezon City 1098
Medford, NJ 1099
Sydney, Australia 1100
Phoenix, AZ  1105
Between Manchester and Lille. 1106
Raleigh 1110
London 1113
Curitiba-PR 1114
|Elsmere| Wilmington, DE. 1115
USA 1121
Chicago 1123
festac,Lagos,Nigeria 1124
USA 1127
Nigeria 1128
Cape Town, South Africa 1129
Virginia 1130
Dunedin, New Zealand 1132
Rome 1134
L.A 1135
North Korea 1136
Glasgow 1138
Hounslow, London 1139
lake worth, Fl 1140
Hollywood 1144
Piedmont, CA 1148
Nashville, Music City, USA 1152
Amsterdam, Worldwide 1153
Detroit, M

Rotterdam, The Netherlands 2014
Philadelphia, PA 2016
Northwest Arkansas 2017
North Carolina, USA 2018
Leesburg, FL 2020
Nashville, Tenn. 2021
Kenya 2022
Kenya 2023
Kenya 2024
Kenya 2025
Kenya 2026
Kenya 2027
Kenya 2028
Kenya 2029
Kenya 2030
Santa Cruz  2034
Kenya 2036
Detroit, Mi 2037
Raleigh, North Carolina, USA 2039
NYC 2045
Albany, Western Australia 2050
U.S. 2054
Islamabad Pakistan 2055
Pittsburgh, PA 2061
Fukushima, JAPAN 2063
Denver, CO 2065
england 2066
Los Angeles via Pittsburgh 2067
Kenya 2068
Perth, Scotland 2079
Waco, Texas 2081
Ohio...yep we said it. 2083
Dili. East Timor 2085
Lynchburg, VA 2086
Mumbai, Maharashtra 2087
Hollywood, Ca 2088
UK 2090
The Woodlands, TX 2092
Central Jersey 2095
Lynchburg, VA 2097
Iowa 2101
North Vernon, IN 2107
Southern California 2110
St. Catherine, Jamaica, W.I. 2111
Pittsburgh, PA 2112
West Coast, Canada 2117
Syracuse, NY 2118
Boca Raton, FL 2124
Columbus, OH 2127
New York 2131
Austin, Texas 2134
United Kingdom 2139
tarn et garonne/lot et gar

In [ ]:
test['']

In [18]:
test.to_csv("csv/test_con_geo.csv")

In [19]:
test.head()

,id,keyword,location,text,loc_nula,loc_valida
0,0,NaN,NaN,Just happened a terrible car crash,1,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1,0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1,0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1,0
